In [78]:
# -*- encoding: utf-8 -*-

import pandas as pd

df = pd.read_csv('salary.csv')

# drop and rename columns
df = df.iloc[:,0:9]
df = df.drop(columns=['职位/职称/注册证', '公积金']) 
df.rename(columns = {'所在市（地区很重要）':'市', '省份':'省'}, inplace = True)

# handle missing data of completely empty rows
df.dropna(axis=0,how='all',inplace=True)

# get overall sense of this dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1659 entries, 0 to 1662
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   省           1640 non-null   object
 1   市           1639 non-null   object
 2   专业          1640 non-null   object
 3   工作年限        1633 non-null   object
 4   学历          1628 non-null   object
 5   年薪(不包括年终奖)  1552 non-null   object
 6   年终奖         1547 non-null   object
dtypes: object(7)
memory usage: 103.7+ KB


quick statistics knowledge:

sampling technique of this shared excel: snowball sampling, which is a type of non-probability sampling, because architects sharing this file to their friends through WeChat.


In [79]:

# oops it's a mess

# what I need for statistical model(multivariate regression):

# aim data type: 省/市/专业/学历 - str(categorical) | 工作年限/年薪(不包括年终奖)/年终奖 - float64

# what I need for machine learning(NLP with TF-IDF and K-means): 备注/弹幕补充ing - str 

# ok let's clean!

# rename
df.rename(columns = {'市':'city', '省':'province', '专业': 'major', '学历':'study_degree', '工作年限':'work_year',  '年薪(不包括年终奖)':'annual_salary','年终奖':'bounce'}, inplace = True)

# reorder: put columns with the same dtype together
df = df[['province', 'city', 'major', 'study_degree', 'work_year', 'annual_salary', 'bounce']]

# infer objects and conver data type
df.convert_dtypes().dtypes

# ahhh need to deal with int64 and float64
import re

# remove all chinese in numeric columns
regex_chinese = re.compile(r'[\u4e00-\u9fff]+', flags=re.IGNORECASE)

num_li = ['annual_salary', 'bounce', 'work_year']
for i in num_li:
    df[i] = df[i].str.replace(regex_chinese,'', regex=True)

# remove all w/ W in annual_salary and bounce
regex_w = re.compile(r'w', flags=re.IGNORECASE)

df['annual_salary']=df['annual_salary'].str.replace(regex_w,'', regex=True)
df['bounce']=df['bounce'].str.replace(regex_w,'', regex=True)

# use the brutal force to convert dtype
def coerce_df_columns_to_numeric(df, column_list):
    df[column_list] = df[column_list].apply(pd.to_numeric, errors='coerce')

coerce_df_columns_to_numeric(df,['work_year','bounce','annual_salary'])

df.head(5)


,province,city,major,study_degree,work_year,annual_salary,bounce
0,宁夏,银川,NaN,本科,1.0,3.6,NaN
1,福建,泉州,建筑,专科,2.0,180.0,NaN
2,福建,福州,结构,本科,NaN,8.1,NaN
3,福建,福州,景观,本科,6.0,13.0,2.0
4,福建,NaN,室内,本科,8.0,NaN,1.0


quick statistics knowledge:

how to analyze categorical data?

ANOVA(A one-way analysis of variance) is used when the categorical variable has at least 3 groups (i.e three different unique values).

What is EDA?

Exploratory Data Analysis (EDA) is an approach to analyze the data using visual techniques. It is used to discover trends, patterns, or to check assumptions with the help of statistical summary and graphical representations.


In [81]:
# EDA 
df.to_csv('semi_structured.csv')

# oops we have outliners!
df.describe()

# let's detect and save/ kill these outliners!
q = df['annual_salary'].quantile(0.99)
d = df[df['annual_salary'] > q]

d

dli = d.index.to_list
dli



<bound method IndexOpsMixin.tolist of Int64Index([125, 127, 141, 142, 197, 233, 329, 708, 1049, 1071, 1072, 1248,
            1654],
           dtype='int64')>